# Create CDAW CME Merge To LASCO Catalog
This notebook is for creating a merged CME/LASCO catalogs of both CME events and time intervals when there are no CMEs (depends on telescope). 

In [1]:
import requests
import pandas as pd

Need_To_Redownload = True

LASCO_Catalog_Filename = '../lasco_img_catalog_lvl_1.txt'
CME_Catalog_Filename = '../cme_event_catalog.txt'
Merged_Catalog_Filename = '../merged_cdaw_lasco_catalog.csv'

# sampling of non-events
C2_non_CME_Catalog_Filename = '../c2_non_events.csv'
C3_non_CME_Catalog_Filename = '../c3_non_events.csv'

In [2]:
# pull the lasco catalog to local disk
if Need_To_Redownload:
    
    # download LASCO catalog now
    lasco_hdr_url = 'https://lasco-www.nrl.navy.mil/lz/img_hdr.txt'
    response = requests.get(lasco_hdr_url, allow_redirects=True)
    
    # cache locally
    open(LASCO_Catalog_Filename, 'wb').write(response.content)


In [3]:
cme_data = pd.read_csv(CME_Catalog_Filename)
cme_data = cme_data.rename(columns={'Unnamed: 0':'CDAW_Id'})
cme_data

,CDAW_Id,Date,Time,Central,Width,Linear,2nd,order,speed,Accel,Mass,Kinetic,MPA,Remarks,Duration_C2,Duration_C3,StartTime_C3_Delta
0,5,1996/01/31,06:52:13,274,47,158,219,99,0,-12.3*,2.5e+14*,3.2e+28*,272,Poor Event; Only 3 points,26582.278481,116518.987342,9746.835443
1,8,1996/02/12,05:47:26,91,53,160,100,211,236,2.0*,1.3e+15,1.6e+29,92,Poor Event,26250.000000,115062.500000,9625.000000
2,12,1996/03/02,04:11:53,88,36,108,95,120,177,1.0*,1.5e+14,8.7e+27,86,Poor Event,38888.888889,170462.962963,14259.259259
3,16,1996/03/07,07:51:56,91,43,60,60,61,75,0.1*,-------,-------,87,Poor Event,70000.000000,306833.333333,25666.666667
4,18,1996/03/09,04:27:49,84,39,93,102,82,0,-1.1*,-------,-------,82,Only 3 points,45161.290323,197956.989247,16559.139785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7610,30475,2020/08/29,00:00:05,86,16,281,220,339,478,8.4*,-------,-------,89,Poor Event,14946.619217,65516.014235,5480.427046
7611,30477,2020/08/29,15:36:05,284,11,196,227,163,0,-4.8*,-------,-------,282,Poor Event,21428.571429,93928.571429,7857.142857
7612,30478,2020/08/29,19:48:05,285,10,218,231,204,79,-2.0*,-------,-------,272,Poor Event,19266.055046,84449.541284,7064.220183
7613,30479,2020/08/30,01:25:44,258,42,117,0,213,286,3.5*,-------,-------,250,Poor Event,35897.435897,157350.427350,13162.393162


In [4]:
# now work to merge in information from LASCO img catalog
lasco_catalog = pd.read_fwf(LASCO_Catalog_Filename, index=False,
                            names=['filename', 'date-obs', 'time-obs','telescope', 'expTime',
                                   'NumCols', 'NumRows', 'StartCol', 'StartRow', 'Filter', 
                                   'Polarizer','Observ.Seq', 'LEB Program', 'junk'])

In [5]:
lasco_catalog

,filename,date-obs,time-obs,telescope,expTime,NumCols,NumRows,StartCol,StartRow,Filter,Polarizer,Observ.Seq,LEB Program,junk
0,12000001.fts,1995/12/08,02:05:12,C1,0.0,512,512,20,1,Ca XV,Clear,Dark,0.0,1.0
1,12000002.fts,1995/12/08,02:12:28,C1,1.9,512,512,20,1,Ca XV,Clear,Dark,0.0,1.0
2,22000001.fts,1995/12/08,02:15:46,C2,0.0,512,512,20,1,Blue,Clear,Dark,0.0,1.0
3,22000002.fts,1995/12/08,02:19:24,C2,1.9,512,512,20,1,Blue,Clear,Dark,0.0,1.0
4,32000001.fts,1995/12/08,02:21:56,C3,0.0,512,512,20,1,Clear,Clear,Dark,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2179785,32648531.fts,2021/01/09,23:18:05,C3,17.6,1024,1024,20,1,Clear,Clear,Normal,0.0,0.0
2179786,22799036.fts,2021/01/09,23:24:05,C2,25.1,1024,1024,20,1,Orange,Clear,Normal,0.0,9.0
2179787,32648532.fts,2021/01/09,23:30:05,C3,17.6,1024,1024,20,1,Clear,Clear,Normal,0.0,0.0
2179788,22799037.fts,2021/01/09,23:36:05,C2,25.1,1024,1024,20,1,Orange,Clear,Normal,0.0,9.0


In [6]:
# Drop C1, EIT observations (this speeds up datetime calc below)
lasco_catalog = lasco_catalog[lasco_catalog['telescope'] != 'C1']
lasco_catalog = lasco_catalog[lasco_catalog['telescope'] != 'EIT']

#date_filt = lasco_catalog[lasco_catalog['date-obs']=='2020/05/28']
#date_filt[date_filt['time-obs']=='21:12:11']
len(lasco_catalog)

1447925

In [7]:
from datetime import datetime, date, time, timezone, timedelta

# add column of python datetime for easier handling
datetime_col = []
for row in lasco_catalog.iterrows():
    date_obs = row[1]['date-obs']
    time_obs = row[1]['time-obs']
    
    date_obs = [int(t) for t in row[1]['date-obs'].split('/')]
    time_obs = [int(t) for t in row[1]['time-obs'].split(':')]
    
    datetime_col.append(datetime(year=date_obs[0], month=date_obs[1], day=date_obs[2], hour=time_obs[0], minute=time_obs[1], second=time_obs[2]))

lasco_catalog['datetime'] = datetime_col
lasco_catalog.index = lasco_catalog.datetime
lasco_catalog

,filename,date-obs,time-obs,telescope,expTime,NumCols,NumRows,StartCol,StartRow,Filter,Polarizer,Observ.Seq,LEB Program,junk,datetime
datetime,,,,,,,,,,,,,,,
1995-12-08 02:15:46,22000001.fts,1995/12/08,02:15:46,C2,0.0,512,512,20,1,Blue,Clear,Dark,0.0,1.0,1995-12-08 02:15:46
1995-12-08 02:19:24,22000002.fts,1995/12/08,02:19:24,C2,1.9,512,512,20,1,Blue,Clear,Dark,0.0,1.0,1995-12-08 02:19:24
1995-12-08 02:21:56,32000001.fts,1995/12/08,02:21:56,C3,0.0,512,512,20,1,Clear,Clear,Dark,0.0,1.0,1995-12-08 02:21:56
1995-12-08 02:25:57,32000002.fts,1995/12/08,02:25:57,C3,1.9,512,512,20,1,Clear,Clear,Dark,0.0,1.0,1995-12-08 02:25:57
1995-12-08 21:43:48,22000003.fts,1995/12/08,21:43:48,C2,2.9,1024,1024,20,1,Lens,Clear,Lamp,0.0,0.0,1995-12-08 21:43:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-09 23:18:05,32648531.fts,2021/01/09,23:18:05,C3,17.6,1024,1024,20,1,Clear,Clear,Normal,0.0,0.0,2021-01-09 23:18:05
2021-01-09 23:24:05,22799036.fts,2021/01/09,23:24:05,C2,25.1,1024,1024,20,1,Orange,Clear,Normal,0.0,9.0,2021-01-09 23:24:05
2021-01-09 23:30:05,32648532.fts,2021/01/09,23:30:05,C3,17.6,1024,1024,20,1,Clear,Clear,Normal,0.0,0.0,2021-01-09 23:30:05


In [8]:
# split up catalog by C2 and C3
c2_lasco = lasco_catalog[lasco_catalog['telescope']=='C2']
c3_lasco = lasco_catalog[lasco_catalog['telescope']=='C3']

In [9]:
# Do the merge of catalogs, using CME events for lookup of filename in LASCO.
# We are not just getting CME images, howerver. For purposes of background subtraction we need to 
# capture the image before first CME event image

cme_ids=[]
filenames=[]
filters = []
dates=[]
times=[]

lasco_keepCols = ['filename', 'telescope', 'NumCols', 'NumRows', 'Filter', 'Polarizer', 'datetime', 'expTime', 'date-obs']
cdaw_keepCols = ['Remarks', 'Central', 'Width', 'Linear', 'CDAW_Id']
bkgnd_keepCols = ['filename', 'NumCols', 'NumRows', 'Filter', 'datetime', 'expTime', 'Polarizer']
bckgnd_fileinfo = [ 'bkgnd_'+colname for colname in bkgnd_keepCols ]

merged_data = {  col:[] for col in lasco_keepCols + cdaw_keepCols + bckgnd_fileinfo }

for event in cme_data.iterrows():
    
    # grab date-time-duration info for our event
    cme_start_date = [int(t) for t in event[1]['Date'].split('/')]
    cme_start_time = [int(t) for t in event[1]['Time'].split(':')]
    cme_c3_delta = timedelta(seconds=event[1]['StartTime_C3_Delta'])
    cme_c2_duration = timedelta(seconds=float(event[1]['Duration_C2']))
    cme_c3_duration = timedelta(seconds=float(event[1]['Duration_C3']))
    
    # convert to Python datetime for easier slicing
    cme_date = date (year=cme_start_date[0], month=cme_start_date[1], day=cme_start_date[2])
    cme_time = time(hour=cme_start_time[0],minute=cme_start_time[1],second=cme_start_time[2])
    
    c2_start = datetime.combine(cme_date, cme_time)
    c2_end   = c2_start + cme_c2_duration
    
    c3_start = c2_start + cme_c3_delta
    c3_end   = c3_start + cme_c3_duration
    
    # get C2 entries
    
    # capture prior image, store with cme image id
    previous_c2_image = c2_lasco[c2_lasco['datetime']<c2_start].iloc[-1]
    
    # and a slice of all of the images which follow
    c2_slice = c2_lasco[c2_lasco['datetime']>c2_start]
    for lasco_row in c2_slice[c2_slice['datetime']<=c2_end].iterrows():
        
        for col in lasco_keepCols:
            merged_data[col].append(lasco_row[1][col])
          
        for col in cdaw_keepCols:
            merged_data[col].append(event[1][col])
        
        #  add in background file to use
        for col in bkgnd_keepCols: 
            merged_data['bkgnd_'+col].append(previous_c2_image[col])
    

    # get C3 entries
    
    # capture prior image, store with cme image id
    previous_c3_image = c3_lasco[c3_lasco['datetime']<c3_start].iloc[-1]
    
    c3_slice = c3_lasco[c3_lasco['datetime']>c3_start]
    for lasco_row in c3_slice[c3_slice['datetime']<=c3_end].iterrows():
        
        for col in lasco_keepCols:
            merged_data[col].append(lasco_row[1][col])
          
        for col in cdaw_keepCols:
            merged_data[col].append(event[1][col])
         
        #  add in background file to use
        for col in bkgnd_keepCols: 
            merged_data['bkgnd_'+col].append(previous_c3_image[col])
    
# convert results to DataFrame
result = pd.DataFrame(merged_data)
result

,filename,telescope,NumCols,NumRows,Filter,Polarizer,datetime,expTime,date-obs,Remarks,...,Width,Linear,CDAW_Id,bkgnd_filename,bkgnd_NumCols,bkgnd_NumRows,bkgnd_Filter,bkgnd_datetime,bkgnd_expTime,bkgnd_Polarizer
0,22000294.fts,C2,1024,1024,Orange,Clear,1996-01-31 08:01:38,25.4,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
1,22000295.fts,C2,1024,1024,Orange,Clear,1996-01-31 09:11:14,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
2,22000296.fts,C2,1024,1024,Orange,Clear,1996-01-31 10:20:39,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
3,22000297.fts,C2,1024,1024,Orange,Clear,1996-01-31 11:30:19,25.9,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
4,22000298.fts,C2,1024,1024,Orange,Clear,1996-01-31 12:40:17,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574731,32633227.fts,C3,1024,1024,Clear,Clear,2020-09-01 03:18:05,17.6,2020/09/01,Poor Event,...,53,213,30480,32645092.fts,1024,1024,Clear,2019-10-30 23:42:05,17.9,Clear
574732,32633228.fts,C3,1024,1024,Clear,Clear,2020-09-01 03:30:05,17.8,2020/09/01,Poor Event,...,53,213,30480,32645092.fts,1024,1024,Clear,2019-10-30 23:42:05,17.9,Clear
574733,32633229.fts,C3,1024,1024,Clear,Clear,2020-09-01 03:42:05,17.6,2020/09/01,Poor Event,...,53,213,30480,32645092.fts,1024,1024,Clear,2019-10-30 23:42:05,17.9,Clear
574734,32633230.fts,C3,1024,1024,Clear,Clear,2020-09-01 03:54:07,17.6,2020/09/01,Poor Event,...,53,213,30480,32645092.fts,1024,1024,Clear,2019-10-30 23:42:05,17.9,Clear


In [10]:
# save the merged catalog
result.to_csv(Merged_Catalog_Filename)

In [11]:
# test it out, try to pull a few intervals for events
C2_data = result[result['telescope']=='C2']
C2_data[:10]

,filename,telescope,NumCols,NumRows,Filter,Polarizer,datetime,expTime,date-obs,Remarks,...,Width,Linear,CDAW_Id,bkgnd_filename,bkgnd_NumCols,bkgnd_NumRows,bkgnd_Filter,bkgnd_datetime,bkgnd_expTime,bkgnd_Polarizer
0,22000294.fts,C2,1024,1024,Orange,Clear,1996-01-31 08:01:38,25.4,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
1,22000295.fts,C2,1024,1024,Orange,Clear,1996-01-31 09:11:14,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
2,22000296.fts,C2,1024,1024,Orange,Clear,1996-01-31 10:20:39,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
3,22000297.fts,C2,1024,1024,Orange,Clear,1996-01-31 11:30:19,25.9,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
4,22000298.fts,C2,1024,1024,Orange,Clear,1996-01-31 12:40:17,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
5,22000299.fts,C2,1024,1024,Orange,Clear,1996-01-31 13:50:01,25.1,1996/01/31,Poor Event; Only 3 points,...,47,158,5,22000292.fts,1024,1024,Orange,1996-01-31 05:42:43,25.1,Clear
26,22000487.fts,C2,1024,1024,Orange,Clear,1996-02-12 07:01:57,25.1,1996/02/12,Poor Event,...,53,160,8,22000485.fts,1024,1024,Orange,1996-02-12 04:32:50,25.1,Clear
27,22000488.fts,C2,1024,1024,Orange,Clear,1996-02-12 08:16:27,25.1,1996/02/12,Poor Event,...,53,160,8,22000485.fts,1024,1024,Orange,1996-02-12 04:32:50,25.1,Clear
28,22000489.fts,C2,1024,1024,Orange,Clear,1996-02-12 09:30:48,25.1,1996/02/12,Poor Event,...,53,160,8,22000485.fts,1024,1024,Orange,1996-02-12 04:32:50,25.1,Clear
29,22000490.fts,C2,1024,1024,Orange,Clear,1996-02-12 10:45:46,25.1,1996/02/12,Poor Event,...,53,160,8,22000485.fts,1024,1024,Orange,1996-02-12 04:32:50,25.1,Clear


In [12]:
#c2_lasco

# now build out the catalog of NON-CME times

In [13]:
# now build out the catalog of NON-CME times
keepCols = ['filename', 'datetime', 'date-obs', 'NumCols', 'NumRows', 'Filter', 'Polarizer', 'expTime', 'telescope']
c2_intervals = c2_lasco[keepCols]
c2_intervals.set_index('datetime')

c3_intervals = c3_lasco[keepCols]
c3_intervals.set_index('datetime')

c2_intervals

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime
datetime,,,,,,,,
1995-12-08 02:15:46,22000001.fts,1995-12-08 02:15:46,1995/12/08,512,512,Blue,Clear,0.0
1995-12-08 02:19:24,22000002.fts,1995-12-08 02:19:24,1995/12/08,512,512,Blue,Clear,1.9
1995-12-08 21:43:48,22000003.fts,1995-12-08 21:43:48,1995/12/08,1024,1024,Lens,Clear,2.9
1995-12-08 22:04:07,22000004.fts,1995-12-08 22:04:07,1995/12/08,1024,1024,Lens,Clear,4.5
1995-12-11 19:55:05,22000005.fts,1995-12-11 19:55:05,1995/12/11,1024,1024,Blue,Clear,2.3
...,...,...,...,...,...,...,...,...
2021-01-09 22:24:05,22799033.fts,2021-01-09 22:24:05,2021/01/09,1024,1024,Orange,Clear,25.1
2021-01-09 22:36:05,22799034.fts,2021-01-09 22:36:05,2021/01/09,1024,1024,Orange,Clear,25.1
2021-01-09 23:12:10,22799035.fts,2021-01-09 23:12:10,2021/01/09,1024,1024,Orange,Clear,25.1


In [24]:
# step through CDAW catalog and calculate when there is NOT an event
    
c2_non_events = { k:[] for k in c2_intervals.columns }
c3_non_events = { k:[] for k in c3_intervals.columns }

sformat = "%Y-%m-%d %H:%M:%S"
    
last_c2_end = None
last_c3_end = None

count = 0
for event in cme_data.iterrows():

    # grab date-time-duration info for our event
    cme_start_date = [int(t) for t in event[1]['Date'].split('/')]
    cme_start_time = [int(t) for t in event[1]['Time'].split(':')]
    cme_c3_delta = timedelta(seconds=event[1]['StartTime_C3_Delta'])
    cme_c2_duration = timedelta(seconds=float(event[1]['Duration_C2']))
    cme_c3_duration = timedelta(seconds=float(event[1]['Duration_C3']))
    
    # convert to Python datetime for easier slicing
    cme_date = date (year=cme_start_date[0], month=cme_start_date[1], day=cme_start_date[2])
    cme_time = time(hour=cme_start_time[0],minute=cme_start_time[1],second=cme_start_time[2])

    c2_start = datetime.combine(cme_date,cme_time)
    c2_end   = c2_start + cme_c2_duration

    c3_start = c2_start + cme_c3_delta
    c3_end   = c3_start + cme_c3_duration
    
    # we have free interval for C2, C3 based on prior event end and 
    # the start of the new event
    if last_c2_end != None:
        for c2_ne in c2_intervals.loc[last_c2_end.strftime(sformat):c2_start.strftime(sformat)].iterrows():
            for col in c2_intervals.columns:
                c2_non_events[col].append(c2_ne[1][col])
                
    if last_c3_end != None:
        for c3_ne in c3_intervals.loc[last_c3_end.strftime(sformat):c3_start.strftime(sformat)].iterrows():
            for col in c3_intervals.columns:
                c3_non_events[col].append(c3_ne[1][col])

    last_c2_end = c2_end
    last_c3_end = c3_end


[1996, 1, 31] 7:23:02.278481
[1996, 2, 12] 7:17:30
[1996, 3, 2] 10:48:08.888889
[1996, 3, 7] 19:26:40
[1996, 3, 9] 12:32:41.290323
[1996, 3, 11] 4:46:53.114754
[1996, 3, 15] 5:15:18.918919
[1996, 4, 8] 9:06:52.500000
[1996, 4, 11] 4:17:21.176471
[1996, 5, 2] 5:12:30
[1996, 5, 14] 3:50:15.789474
[1996, 5, 16] 3:45:48.387097
[1996, 5, 25] 6:08:25.263158
[1996, 6, 17] 3:50:15.789474
[1996, 7, 1] 3:14:59.164345
[1996, 7, 8] 4:25:09.090909
[1996, 7, 12] 1:04:30.967742
[1996, 7, 15] 1:50:45.569620
[1996, 7, 15] 4:51:40
[1996, 7, 17] 3:38:45
[1996, 7, 17] 3:45:48.387097
[1996, 7, 20] 4:44:33.170732
[1996, 7, 28] 9:15:33.333333
[1996, 8, 3] 16:25:54.929577
[1996, 8, 10] 7:52:58.378378
[1996, 8, 14] 4:46:53.114754
[1996, 9, 2] 2:02:35.516637
[1996, 9, 4] 4:34:30.588235
[1996, 9, 9] 7:59:27.123288
[1996, 9, 20] 9:06:52.500000
[1996, 9, 22] 6:12:20.425532
[1996, 10, 1] 3:38:45
[1996, 10, 5] 2:03:01.370826
[1996, 10, 8] 7:34:32.727273
[1996, 10, 9] 3:54:06.822742
[1996, 10, 11] 5:34:55.693780
[199

[1998, 11, 5] 2:03:14.366197
[1998, 11, 6] 2:52:50.370370
[1998, 11, 7] 2:55:52.763819
[1998, 11, 7] 1:50:45.569620
[1998, 11, 8] 2:05:13.416816
[1998, 11, 9] 3:35:23.076923
[1998, 11, 9] 4:13:37.391304
[1998, 11, 10] 4:04:45.314685
[1998, 11, 10] 4:32:22.412451
[1998, 11, 11] 1:54:11.549755
[1998, 11, 11] 3:01:49.090909
[1998, 11, 28] 2:08:12.307692
[1998, 12, 4] 1:41:53.537118
[1998, 12, 10] 1:58:26.598985
[1998, 12, 17] 3:51:47.284768
[1998, 12, 20] 3:12:50.247934
[1998, 12, 20] 1:33:05.106383
[1999, 2, 6] 8:34:42.352941
[1999, 2, 7] 2:59:01.687980
[1999, 2, 8] 1:20:33.141542
[1999, 2, 9] 1:22:03.798359
[1999, 2, 9] 1:26:38.019802
[1999, 2, 10] 3:06:10.212766
[1999, 2, 12] 1:35:53.424658
[1999, 2, 22] 3:44:21.538462
[1999, 3, 1] 1:51:38.564593
[1999, 3, 1] 3:13:54.349030
[1999, 3, 1] 1:33:42.489960
[1999, 3, 2] 1:42:02.448980
[1999, 3, 7] 4:34:30.588235
[1999, 3, 9] 1:42:29.341142
[1999, 3, 10] 1:50:56.101426
[1999, 3, 11] 4:49:15.371901
[1999, 3, 13] 3:21:08.965517
[1999, 3, 14] 4:

[1999, 10, 14] 2:04:06.808511
[1999, 10, 16] 4:50:27.385892
[1999, 10, 16] 3:18:51.818182
[1999, 10, 17] 3:51:47.284768
[1999, 10, 18] 4:43:24.048583
[1999, 10, 18] 8:06:06.666667
[1999, 10, 18] 2:31:11.274298
[1999, 10, 19] 1:32:57.689243
[1999, 10, 22] 3:14:59.164345
[1999, 10, 23] 1:09:10.197628
[1999, 10, 23] 3:08:40.754717
[1999, 10, 23] 4:07:20.989399
[1999, 10, 24] 2:25:50
[1999, 10, 25] 2:16:59.178082
[1999, 10, 25] 1:30:05.405405
[1999, 10, 27] 4:40:00
[1999, 10, 27] 4:57:52.340426
[1999, 10, 28] 2:21:42.024291
[1999, 10, 28] 1:41:44.651163
[1999, 10, 30] 6:10:22.222222
[1999, 10, 31] 2:45:29.078014
[1999, 10, 31] 2:28:18.305085
[1999, 10, 31] 1:59:39.487179
[1999, 10, 31] 1:44:19.314456
[1999, 11, 2] 5:08:22.202643
[1999, 11, 5] 3:05:11.111111
[1999, 11, 6] 2:51:34.117647
[1999, 11, 7] 1:12:18.842975
[1999, 11, 7] 2:07:16.363636
[1999, 11, 7] 2:27:03.529412
[1999, 11, 7] 4:57:52.340426
[1999, 11, 8] 7:34:32.727273
[1999, 11, 8] 5:25:34.883721
[1999, 11, 8] 3:14:59.164345
[199

[2000, 8, 18] 1:18:02.274247
[2000, 8, 18] 3:04:12.631579
[2000, 8, 19] 2:34:11.101322
[2000, 8, 19] 3:42:13.333333
[2000, 8, 20] 2:01:06.435986
[2000, 8, 21] 2:35:12.638581
[2000, 8, 21] 3:10:44.141689
[2000, 8, 24] 1:40:43.165468
[2000, 8, 25] 2:15:08.108108
[2000, 8, 26] 2:10:06.691450
[2000, 8, 26] 3:34:43.435583
[2000, 8, 27] 2:32:30.326797
[2000, 8, 27] 4:32:22.412451
[2000, 8, 28] 2:09:09.077491
[2000, 8, 29] 1:29:17.142857
[2000, 8, 29] 2:15:39.534884
[2000, 8, 29] 1:31:01.638492
[2000, 8, 30] 4:44:33.170732
[2000, 8, 31] 1:28:36.455696
[2000, 9, 1] 1:56:05.174129
[2000, 9, 1] 2:43:56.065574
[2000, 9, 2] 6:10:22.222222
[2000, 9, 2] 1:44:47.425150
[2000, 9, 3] 1:50:24.605678
[2000, 9, 3] 3:18:18.016997
[2000, 9, 4] 1:22:26.996466
[2000, 9, 4] 3:51:01.386139
[2000, 9, 5] 2:27:59.492600
[2000, 9, 6] 1:48:31.627907
[2000, 9, 6] 2:11:49.604520
[2000, 9, 6] 2:20:00
[2000, 9, 6] 2:07:44.233577
[2000, 9, 6] 3:14:59.164345
[2000, 9, 7] 2:03:14.366197
[2000, 9, 7] 2:24:55.652174
[2000, 9

[2001, 3, 27] 4:18:18.154982
[2001, 3, 27] 3:25:52.941176
[2001, 3, 29] 2:13:04.790875
[2001, 3, 31] 2:37:18.202247
[2001, 3, 31] 2:18:04.023669
[2001, 4, 2] 1:35:45.554036
[2001, 4, 2] 0:27:56.646707
[2001, 4, 3] 0:43:23.843769
[2001, 4, 3] 1:08:17.560976
[2001, 4, 4] 1:36:17.166437
[2001, 4, 5] 0:40:00
[2001, 4, 7] 0:31:50.828025
[2001, 4, 9] 1:47:11.852986
[2001, 4, 14] 1:41:09.364162
[2001, 4, 14] 2:28:18.305085
[2001, 4, 14] 3:54:06.822742
[2001, 4, 14] 1:31:37.382199
[2001, 4, 14] 3:42:13.333333
[2001, 4, 15] 0:58:22.919099
[2001, 4, 19] 2:58:34.285714
[2001, 4, 20] 1:00:20.689655
[2001, 4, 21] 1:12:27.826087
[2001, 4, 22] 2:19:26.533865
[2001, 4, 27] 3:42:13.333333
[2001, 4, 30] 2:06:34.936709
[2001, 5, 1] 2:45:05.660377
[2001, 5, 1] 2:37:39.459459
[2001, 5, 2] 5:05:40.611354
[2001, 5, 3] 5:15:18.918919
[2001, 5, 3] 4:20:13.382900
[2001, 5, 4] 5:38:09.855072
[2001, 5, 5] 3:02:17.500000
[2001, 5, 7] 1:55:53.642384
[2001, 5, 7] 0:57:14.178250
[2001, 5, 8] 2:14:36.923077
[2001, 5, 

...

[2014, 12, 10] 9:15:33.333333
[2014, 12, 10] 2:01:06.435986
[2014, 12, 11] 4:44:33.170732
[2014, 12, 11] 2:12:34.545455
[2014, 12, 12] 1:01:46.972639
[2014, 12, 13] 4:32:22.412451
[2014, 12, 13] 4:44:33.170732
[2014, 12, 13] 3:17:10.985915
[2014, 12, 14] 2:54:07.761194
[2014, 12, 14] 4:42:15.483871
[2014, 12, 14] 3:07:40.053619
[2014, 12, 14] 2:01:06.435986
[2014, 12, 14] 1:51:49.265176
[2014, 12, 15] 3:30:50.602410
[2014, 12, 15] 6:10:22.222222
[2014, 12, 16] 3:21:43.746398
[2014, 12, 19] 2:33:10.371991
[2014, 12, 19] 2:36:57.040359
[2014, 12, 20] 4:04:45.314685
[2014, 12, 20] 1:24:20.240964
[2014, 12, 25] 1:42:11.386861
[2014, 12, 25] 1:51:38.564593
[2014, 12, 26] 1:03:48.623519
[2014, 12, 26] 5:01:43.448276
[2014, 12, 26] 2:33:10.371991
[2014, 12, 29] 3:03:43.622047
[2014, 12, 29] 2:09:37.777778
[2014, 12, 29] 3:51:01.386139
[2014, 12, 29] 4:18:18.154982
[2014, 12, 30] 1:20:10.996564
[2014, 12, 30] 8:38:31.111111
[2014, 12, 30] 1:54:56.551724
[2014, 12, 31] 7:12:05.925926
[2014, 12,

[2015, 5, 22] 5:31:45.213270
[2015, 5, 22] 3:25:16.715543
[2015, 5, 23] 4:48:03.950617
[2015, 5, 23] 4:15:28.467153
[2015, 5, 26] 7:25:51.592357
[2015, 5, 27] 4:41:07.469880
[2015, 5, 27] 4:54:07.058824
[2015, 5, 27] 2:55:00
[2015, 5, 28] 7:17:30
[2015, 5, 28] 2:50:18.978102
[2015, 5, 29] 7:37:30.980392
[2015, 5, 30] 11:13:04.615385
[2015, 6, 1] 5:18:10.909091
[2015, 6, 1] 4:57:52.340426
[2015, 6, 2] 2:41:17.419355
[2015, 6, 2] 3:30:12.612613
[2015, 6, 2] 3:54:06.822742
[2015, 6, 2] 5:55:19.796954
[2015, 6, 3] 5:16:44.524887
[2015, 6, 3] 2:08:40.588235
[2015, 6, 4] 5:44:49.655172
[2015, 6, 4] 4:46:53.114754
[2015, 6, 4] 1:47:11.852986
[2015, 6, 5] 4:56:36.610169
[2015, 6, 5] 3:55:41.414141
[2015, 6, 7] 2:45:52.606635
[2015, 6, 7] 2:28:18.305085
[2015, 6, 9] 3:04:12.631579
[2015, 6, 9] 1:07:34.054054
[2015, 6, 10] 2:40:33.027523
[2015, 6, 10] 21:12:43.636364
[2015, 6, 11] 1:41:35.791001
[2015, 6, 11] 2:45:29.078014
[2015, 6, 14] 0:57:00.195440
[2015, 6, 14] 2:46:40
[2015, 6, 14] 3:53:20

[2015, 11, 7] 4:48:03.950617
[2015, 11, 8] 2:20:16.833667
[2015, 11, 8] 15:09:05.454545
[2015, 11, 9] 1:07:14.582133
[2015, 11, 10] 3:38:45
[2015, 11, 10] 2:44:19.154930
[2015, 11, 11] 1:51:17.265501
[2015, 11, 11] 2:31:30.909091
[2015, 11, 12] 3:01:20.829016
[2015, 11, 12] 3:23:29.302326
[2015, 11, 13] 3:17:10.985915
[2015, 11, 13] 2:51:59.410319
[2015, 11, 14] 3:04:41.794195
[2015, 11, 15] 3:10:44.141689
[2015, 11, 15] 4:18:18.154982
[2015, 11, 15] 2:19:43.233533
[2015, 11, 16] 1:22:03.798359
[2015, 11, 16] 3:22:18.728324
[2015, 11, 16] 3:51:47.284768
[2015, 11, 17] 4:50:27.385892
[2015, 11, 17] 3:53:20
[2015, 11, 18] 2:59:56.915167
[2015, 11, 18] 4:10:00
[2015, 11, 18] 1:13:04.133612
[2015, 11, 19] 5:16:44.524887
[2015, 11, 21] 3:51:01.386139
[2015, 11, 24] 9:24:30.967742
[2015, 11, 25] 5:00:25.751073
[2015, 11, 26] 2:38:22.262443
[2015, 11, 27] 3:24:04.897959
[2015, 11, 27] 4:11:47.913669
[2015, 11, 28] 10:42:12.110092
[2015, 11, 30] 2:19:26.533865
[2015, 11, 30] 3:57:17.288136
[20

[2016, 7, 26] 3:59:43.561644
[2016, 7, 27] 2:08:54.806630
[2016, 7, 28] 3:32:45.957447
[2016, 7, 28] 7:34:32.727273
[2016, 7, 28] 4:24:09.056604
[2016, 7, 30] 4:00:32.989691
[2016, 7, 30] 4:02:12.871972
[2016, 8, 1] 6:16:20.645161
[2016, 8, 7] 2:07:02.504537
[2016, 8, 7] 4:59:08.717949
[2016, 8, 8] 4:02:12.871972
[2016, 8, 8] 2:55:52.763819
[2016, 8, 9] 9:06:52.500000
[2016, 8, 9] 3:04:12.631579
[2016, 8, 10] 5:01:43.448276
[2016, 8, 11] 3:12:50.247934
[2016, 8, 12] 9:43:20
[2016, 8, 12] 3:20:00
[2016, 8, 13] 2:55:52.763819
[2016, 8, 13] 1:58:26.598985
[2016, 8, 14] 3:41:31.139241
[2016, 8, 14] 18:13:45
[2016, 8, 16] 4:20:13.382900
[2016, 8, 17] 4:40:00
[2016, 8, 17] 5:41:27.804878
[2016, 8, 18] 1:24:08.076923
[2016, 8, 18] 4:18:18.154982
[2016, 8, 19] 17:24:46.567164
[2016, 8, 19] 4:49:15.371901
[2016, 8, 19] 3:28:20
[2016, 8, 22] 4:21:11.641791
[2016, 8, 22] 5:18:10.909091
[2016, 8, 26] 9:33:46.229508
[2016, 8, 29] 6:28:53.333333
[2016, 8, 30] 2:01:31.666667
[2016, 8, 30] 5:43:08.235

[2017, 11, 21] 5:46:32.079208
[2017, 11, 28] 3:42:13.333333
[2017, 12, 1] 3:41:31.139241
[2017, 12, 1] 2:23:08.957055
[2017, 12, 3] 4:21:11.641791
[2017, 12, 5] 3:12:50.247934
[2017, 12, 9] 4:20:13.382900
[2017, 12, 14] 5:58:58.461538
[2017, 12, 19] 5:38:09.855072
[2017, 12, 21] 5:05:40.611354
[2017, 12, 27] 2:51:34.117647
[2017, 12, 28] 2:46:40
[2018, 1, 3] 4:46:53.114754
[2018, 1, 4] 4:50:27.385892
[2018, 1, 10] 6:18:22.702703
[2018, 1, 12] 4:57:52.340426
[2018, 1, 13] 4:01:22.758621
[2018, 1, 22] 3:57:17.288136
[2018, 1, 23] 5:21:06.055046
[2018, 1, 28] 4:32:22.412451
[2018, 2, 4] 8:02:45.517241
[2018, 2, 8] 5:01:43.448276
[2018, 2, 17] 5:48:15.522388
[2018, 3, 1] 5:46:32.079208
[2018, 3, 11] 6:16:20.645161
[2018, 3, 12] 6:46:58.604651
[2018, 3, 15] 4:59:08.717949
[2018, 3, 25] 4:00:32.989691
[2018, 3, 27] 5:46:32.079208
[2018, 3, 28] 6:00:49.484536
[2018, 3, 29] 8:42:23.283582
[2018, 3, 30] 4:15:28.467153
[2018, 3, 30] 4:27:10.534351
[2018, 3, 31] 3:56:29.189189
[2018, 3, 31] 4:27:

In [25]:
c2_intervals[300:310]

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime
datetime,,,,,,,,
1996-02-01 00:43:07,22000301.fts,1996-02-01 00:43:07,1996/02/01,1024,1024,Orange,Clear,25.1
1996-02-01 01:07:04,22000302.fts,1996-02-01 01:07:04,1996/02/01,1024,1024,Orange,Clear,25.1
1996-02-01 02:21:56,22000303.fts,1996-02-01 02:21:56,1996/02/01,1024,1024,Orange,Clear,25.0
1996-02-01 03:36:53,22000304.fts,1996-02-01 03:36:53,1996/02/01,1024,1024,Orange,Clear,25.1
1996-02-01 04:51:44,22000305.fts,1996-02-01 04:51:44,1996/02/01,1024,1024,Orange,Clear,25.1
1996-02-01 06:06:38,22000306.fts,1996-02-01 06:06:38,1996/02/01,1024,1024,Orange,Clear,25.1
1996-02-01 07:21:32,22000307.fts,1996-02-01 07:21:32,1996/02/01,1024,1024,Orange,Clear,25.1
1996-02-01 08:36:25,22000308.fts,1996-02-01 08:36:25,1996/02/01,1024,1024,Orange,Clear,25.1
1996-02-01 09:51:11,22000309.fts,1996-02-01 09:51:11,1996/02/01,1024,1024,Orange,Clear,25.1


In [26]:
c2_intervals.loc['1996-01-31 00:00:00':'2000-02-12 00:00:00']

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime
datetime,,,,,,,,
1996-01-31 01:55:44,22000288.fts,1996-01-31 01:55:44,1996/01/31,1024,1024,Orange,Clear,25.1
1996-01-31 02:20:38,22000289.fts,1996-01-31 02:20:38,1996/01/31,1024,1024,Orange,Clear,25.1
1996-01-31 03:24:01,22000290.fts,1996-01-31 03:24:01,1996/01/31,1024,1024,Orange,Clear,13.3
1996-01-31 04:32:28,22000291.fts,1996-01-31 04:32:28,1996/01/31,1024,1024,Orange,Clear,25.1
1996-01-31 05:42:43,22000292.fts,1996-01-31 05:42:43,1996/01/31,1024,1024,Orange,Clear,25.1
...,...,...,...,...,...,...,...,...
1999-12-11 00:04:02,22057390.fts,1999-12-11 00:04:02,1999/12/11,512,512,Orange,-60 D,37.8
1999-12-11 00:09:54,22057391.fts,1999-12-11 00:09:54,1999/12/11,1024,1024,Orange,Clear,62.0
1999-12-11 00:18:04,22057392.fts,1999-12-11 00:18:04,1999/12/11,1024,1024,Orange,+60 D,54.0


In [27]:
c2_non_events = pd.DataFrame(c2_non_events)
c2_non_events

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime
0,22000300.fts,1996-01-31 18:48:55,1996/01/31,512,512,Orange,Clear,6.4
1,22000301.fts,1996-02-01 00:43:07,1996/02/01,1024,1024,Orange,Clear,25.1
2,22000302.fts,1996-02-01 01:07:04,1996/02/01,1024,1024,Orange,Clear,25.1
3,22000303.fts,1996-02-01 02:21:56,1996/02/01,1024,1024,Orange,Clear,25.0
4,22000304.fts,1996-02-01 03:36:53,1996/02/01,1024,1024,Orange,Clear,25.1
...,...,...,...,...,...,...,...,...
675653,22782334.fts,2020-08-31 01:25:39,2020/08/31,1024,1024,Orange,Clear,25.1
675654,22782335.fts,2020-08-31 01:36:11,2020/08/31,1024,1024,Orange,Clear,25.1
675655,22782336.fts,2020-08-31 01:48:05,2020/08/31,1024,1024,Orange,Clear,26.2
675656,22782337.fts,2020-08-31 02:00:05,2020/08/31,1024,1024,Orange,Clear,25.1


In [28]:
c3_non_events = pd.DataFrame(c3_non_events)
c3_non_events

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime
0,32003726.fts,1996-02-02 02:26:29,1996/02/02,1024,1024,Clear,Clear,19.1
1,32003727.fts,1996-02-02 03:02:49,1996/02/02,1024,1024,Clear,Clear,10.3
2,32003728.fts,1996-02-02 04:10:18,1996/02/02,1024,1024,Clear,Clear,19.1
3,32003729.fts,1996-02-02 05:17:45,1996/02/02,1024,1024,Clear,Clear,19.9
4,32003730.fts,1996-02-02 06:24:17,1996/02/02,1024,1024,Clear,Clear,19.1
...,...,...,...,...,...,...,...,...
360274,32632880.fts,2020-08-29 00:18:05,2020/08/29,1024,1024,Clear,Clear,17.6
360275,32632881.fts,2020-08-29 00:30:05,2020/08/29,1024,1024,Clear,Clear,17.6
360276,32632882.fts,2020-08-29 00:42:05,2020/08/29,1024,1024,Clear,Clear,17.7
360277,32632883.fts,2020-08-29 00:54:05,2020/08/29,1024,1024,Clear,Clear,17.7


In [29]:
# save it 
c2_non_events.to_csv(C2_non_CME_Catalog_Filename, index=False)
c3_non_events.to_csv(C3_non_CME_Catalog_Filename, index=False)

In [20]:
event_catalog  = result
test_time = event_catalog['datetime'][1]
test_time

Timestamp('1996-01-31 09:11:14')

In [21]:
len(event_catalog), len(c2_intervals)

(574736, 799219)

In [22]:
# build out a filtered catlog so that we match only stuff we can accept
# for differencing later on (e.g. same params as before)
c2_a = c2_intervals[c2_intervals['expTime']>24.]
c2_b = c2_a[c2_a['expTime']<27.]
c2_c = c2_b[c2_b['Filter'].str.match('Orang')]
c2_d = c2_c[c2_c['NumCols'] == 1024]
c2_filt_cat = c2_d[c2_d['NumRows'] == 1024]

c2_filt_cat

,filename,datetime,date-obs,NumCols,NumRows,Filter,Polarizer,expTime
datetime,,,,,,,,
1996-01-28 17:18:10,22000254.fts,1996-01-28 17:18:10,1996/01/28,1024,1024,Orange,Clear,25.1
1996-01-30 03:54:18,22000278.fts,1996-01-30 03:54:18,1996/01/30,1024,1024,Orange,Clear,25.0
1996-01-30 04:38:27,22000279.fts,1996-01-30 04:38:27,1996/01/30,1024,1024,Orange,Clear,25.1
1996-01-30 05:56:07,22000280.fts,1996-01-30 05:56:07,1996/01/30,1024,1024,Orange,Clear,25.1
1996-01-30 07:13:53,22000281.fts,1996-01-30 07:13:53,1996/01/30,1024,1024,Orange,Clear,25.1
...,...,...,...,...,...,...,...,...
2021-01-09 22:24:05,22799033.fts,2021-01-09 22:24:05,2021/01/09,1024,1024,Orange,Clear,25.1
2021-01-09 22:36:05,22799034.fts,2021-01-09 22:36:05,2021/01/09,1024,1024,Orange,Clear,25.1
2021-01-09 23:12:10,22799035.fts,2021-01-09 23:12:10,2021/01/09,1024,1024,Orange,Clear,25.1


In [23]:
d = c2_filt_cat[c2_filt_cat['datetime']<test_time]
foo = d.iloc[0]
type(foo), foo

(pandas.core.series.Series,
 filename            22000254.fts
 datetime     1996-01-28 17:18:10
 date-obs              1996/01/28
 NumCols                     1024
 NumRows                     1024
 Filter                    Orange
 Polarizer                  Clear
 expTime                     25.1
 Name: 1996-01-28 17:18:10, dtype: object)